Food Recommender System
==

Reference site : https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101

This Notebook is document for developing Food recommender system for the ones having hard time deciding what menu they would eat.

Contributor : Taenyun Kim



Things to Develop (After midterm...)
--


1. function to save **df_user_info**, **interactions_full_df**, **food_df** into .csv file.

1. function to recommend different items each round.

1. function to update food ratings.

1. fucntion tocrawl recipes from the web for the content-based filtering... *(might not be needed? but I personally think that it is better with having both hash tag and web-crawled recipe data into the model for the better performanaces....)*

1. hybrid recommender system.

1. create knowledge based filtering for mood-and-weather-based recommender

1. function to extract *n* food items that follow gaussian distribution and make new users to rate for solving cold start problem.

In [1]:
#import modules
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
from scipy.sparse.linalg import svds
from itertools import combinations 
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

In [3]:
from GoEatWebCrawler import recipe_finder

Import Data
---

In [9]:
#import food rating data
df_food1 = pd.read_csv('data\\food_rating.csv')
df_food2 = pd.read_csv('data\\food_rating2.csv')

In [10]:
#set user id
df_food_reset_index1 = df_food.reset_index()
df_food_reset_index2 = df_food2.reset_index()

In [11]:
df_food_reset_index2['index'] = df_food_reset_index2['index'] + len(df_food_reset_index['index'])

Divide Data into User information, and Rating
--

In [12]:
user_info_index1 = [0,1,2,3,4]  + list(range(54,58))
user_info_index2 = [0,1,2,3,4]  + list(range(65,69))
food_drop_index1 = list(range(0,5)) + list(range(54,58))
food_drop_index2 = list(range(0,5)) + list(range(65,69))
user_detail_index = ['aloneHow','eatAlone','eatDate','eatTogether']


In [53]:
df_user_info_original1 = df_food_reset_index.iloc[:,user_info_index1]
df_user_info_original2 = df_food_reset_index2.iloc[:,user_info_index2]

df_food_rating1 = df_food_reset_index1.drop(axis =1,columns=df_food_reset_index1.columns[food_drop_index1])
df_food_rating2 = df_food_reset_index2.drop(axis =1,columns=df_food_reset_index2.columns[food_drop_index2])

In [54]:
df_user_info_original1.columns = ['userId','timestamp','userName','sex','age','aloneHow','eatAlone','eatDate','eatTogether']
df_user_info_original2.columns = ['userId','timestamp','userName','sex','age','aloneHow','eatAlone','eatDate','eatTogether']
#userId : 사용자 고유번호
#timestamp : 설문시각
#userName : 사용자 이름
#sex : 성별
#age : 나이
#aloneHow : 혼밥은 주로 어떻게 하나요?
#eatAlone : 혼밥할 때 주로 먹는 음식 메뉴는 무엇인가요? 한 가지만 적어주세요.
#eatDate : (선택사항) 데이트를 하는 상황에서는 주로 무슨 음식을 즐겨 먹나요? 한 가지만 적어주세요.
#eatTogether : 친한 친구들 여럿이서 만나는 자리에서는 무슨 음식을 즐겨 먹나요? 한 가지만 적어 주세요.

In [55]:
df_user_detail1 = df_user_info_original.loc[:,['userId']+user_detail_index]
df_user_detail2 = df_user_info_original2.loc[:,['userId']+user_detail_index]

df_user_detail = df_user_detail1.append(df_user_detail2,ignore_index = True)

df_user_info1 = df_user_info_original1.drop(user_detail_index, axis=1)
df_user_info2 = df_user_info_original2.drop(user_detail_index, axis=1)

df_user_info = df_user_info1.append(df_user_info2,ignore_index = True)


In [56]:
drop_foods = df_food_rating1.columns[[0,1,2,9,10]]

In [57]:
df_food_rating1 = df_food_rating1.drop(axis=1,columns = drop_foods )

In [58]:
df_food_rating2 = df_food_rating2.drop(axis=1, columns = ['만두.1','파타이'])

In [59]:
df_food_rating1.columns = ['볶음밥', '김치찌개', '된장찌개', '닭갈비', '닭도리탕', '불고기', '삼겹살', '수육', '제육볶음',
       '족발', '잔치국수', '칼국수', '콩국수', '비빔밥', '삼계탕', '게장', '수제비', '초밥',
       '돈부리', '돈까스', '회', '라멘', '냉모밀', '우동', '스테이크', '리조또', '파스타',
       '함박스테이크', '피자', '짜장면', '짬뽕', '샤브샤브', '쌀국수', '월남쌈', '커리', '김밥',
       '떡볶이', '라면', '오므라이스', '컵밥', '브리또', '햄버거', '샌드위치', '치킨']

df_food_rating1['타코'] = df_food_rating1['브리또']

In [60]:
df_food_rating2.columns = ['갈비찜', '찜닭', '갈비탕', '설렁탕', '막창', '곱창', '볶음밥', '김치찌개', '된장찌개', '닭도리탕',
       '불고기', '물냉면', '비빔냉면', '뼈해장국', '순대국', '전', '소고기', '갈비', '오리고기', '양고기',
       '팟타이', '제육볶음', '족발', '잔치국수', '칼국수', '생선구이', '알탕', '삼계탕', '게장', '떡갈비',
       '돈부리', '쭈꾸미', '돈까스', '라멘', '냉모밀', '우동', '만두', '리조또', '샐러드',
       '아구찜', '쌈밥', '짜장면', '짬뽕', '토스트', '샤브샤브', '부대찌개', '똠양꿍', '커리', '매운탕',
       '떡볶이', '순대', '라면', '오므라이스', '컵밥', '브리또', '햄버거', '청국장', '마라탕']

df_food_rating2['타코'] = df_food_rating2['브리또']

In [61]:
df_food_rating_stack1 = pd.DataFrame(df_food_rating1.stack()).reset_index() 
df_food_rating_stack2 = pd.DataFrame(df_food_rating2.stack()).reset_index()
df_food_rating_stack2['level_0'] = df_food_rating_stack2['level_0'] + len(df_food_reset_index['index'])

df_food_rating_stack1.columns =['userId','foodId','eventStrength']
df_food_rating_stack2.columns =['userId','foodId','eventStrength']
df_food_rating_stack = df_food_rating_stack1.append(df_food_rating_stack2)

In [62]:
df_food_rating_stack

,userId,foodId,eventStrength
0,0,볶음밥,3
1,0,김치찌개,2
2,0,된장찌개,4
3,0,닭갈비,3
4,0,닭도리탕,1
5,0,불고기,3
6,0,삼겹살,4
7,0,수육,5
8,0,제육볶음,2
9,0,족발,4


추천 시스템을 위한 변수 새롭게 설정
--

In [63]:
#food_df 
#foodId : 음식 고유번호 
#FoodName : 고유번호에 따른 음식 이름

food_df = pd.DataFrame(df_food_rating_stack.foodId.unique()).reset_index()
food_df.columns = ['foodId','foodName'] 

In [65]:
#interactions_full_df 
#userId : 개인 고유번호
#foodId : 음식 고유번호 
#eventStrength : 음식에 대한 평가

interactions_full_with_zeros_df = df_food_rating_stack.copy().reset_index(drop=True)

for food in range(len(food_df.foodName)):
    interactions_full_with_zeros_df.loc[interactions_full_with_zeros_df.foodId == food_df.foodName[food],'foodId'] = food_df.foodId[food]
interactions_df  = interactions_full_with_zeros_df[interactions_full_with_zeros_df['eventStrength'] != 0]



데이터 프레임 저장
--

In [69]:
df_user_detail.iloc[205,2] = '햄버거'

In [70]:
df_user_info.to_csv('data\\info\\df_user_info.csv',index=False,encoding = 'cp949')
df_user_detail.to_csv('data\\info\\df_user_detail.csv',index=False,encoding = 'cp949')
food_df.to_csv('data\\info\\food_df.csv',index=False,encoding = 'cp949')
interactions_df.to_csv('data\\info\\interactions_df.csv',index=False,encoding = 'cp949')


User with at least 5 interaction
--
Recommender systems have a problem known as user cold-start, in which is hard do provide personalized recommendations for users with none or a very few number of consumed items, due to the lack of information to model their preferences.
For this reason, we are keeping in the dataset only users with at leas 5 interactions.

In [13]:
users_interactions_count_df = interactions_df.groupby(['userId', 'foodId']).size().groupby('userId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['userId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 159
# users with at least 5 interactions: 159


In [14]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'userId',
               right_on = 'userId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 7693
# of interactions from users with at least 5 interactions: 7693


In [15]:
#Users are allowed to rate an food many times, and interact with them in different ratings. 
#Thus, to model the user interest on a given food, we aggregate all the ratings the user has performed 
#in an item by a weighted sum of rating score strength and apply a log transformation to smooth the distribution.

#shoudn't be mean??

def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['userId', 'foodId'])['eventStrength'].mean() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 7693


,userId,foodId,eventStrength
0,0,0,2.321928
1,0,1,2.000000
2,0,2,1.000000
3,0,3,1.584963
4,0,4,2.321928
5,0,5,2.584963
6,0,6,2.321928
7,0,7,2.321928
8,0,8,2.321928
9,0,9,2.584963


Model Evaluation
--

Evaluation is important for machine learning projects, because it allows to compare objectivelly different algorithms and hyperparameter choices for models.
One key aspect of evaluation is to ensure that the trained model generalizes for data it was not trained on, using Cross-validation techniques. We are using here a simple cross-validation approach named holdout, in which a random data sample (20% in this case) are kept aside in the training process, and exclusively used for evaluation. All evaluation metrics reported here are computed using the test set.

Ps. **A more robust evaluation approach** could be to split train and test sets by a **reference date**, where **the train set** is composed by **all interactions before that date**, and **the test set** are interactions **after that date**. For the sake of simplicity, we chose the first random approach for this notebook, but you may want to try the second approach to better simulate how the recsys would perform in production predicting "future" users interactions.

In [16]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['userId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 6154
# interactions on Test set: 1539


In [17]:
#Indexing by userId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('userId')
interactions_train_indexed_df = interactions_train_df.set_index('userId')
interactions_test_indexed_df = interactions_test_df.set_index('userId')

In [18]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['foodId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [19]:
#Top-N accuracy metrics consts

#EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 0

class ModelEvaluator:
    
    def set_random_sample_non_interacted(self,EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS):
        self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(food_df['foodId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['foodId']) == pd.core.series.Series:
            person_interacted_items_testset = set(interacted_values_testset['foodId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['foodId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32)) ## seed=item_id%(2**32)
            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['foodId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['foodId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()   

Popularity Based Recommneder
==

In [20]:
df_food_grouped = interactions_full_df.groupby(['foodId']).agg({'eventStrength': 'sum'}).reset_index()
grouped_rating = interactions_full_df.groupby(['foodId']).agg({'userId': 'count'}).reset_index()
df_food_grouped['eventStrength']  = df_food_grouped['eventStrength'].div(grouped_rating['userId'])
df_food_ranking = df_food_grouped.sort_values(['eventStrength', 'foodId'], ascending = [0,1])

df_food_ranking['foodName'] = ''

for food in range(len(food_df.foodId)):
    df_food_ranking.loc[df_food_ranking.foodId == food_df.foodId[food],'foodName'] = food_df.foodName[food]

df_food_ranking.index = range(1,len(df_food_ranking)+1)

Most Favored 10 Foods
--

In [21]:
df_food_ranking.head(10)

,foodId,eventStrength,foodName
1,6,2.401470,삼겹살(구이)
2,21,2.396908,초밥
3,39,2.393494,스테이크
4,24,2.378197,치킨
5,12,2.344726,파스타
6,43,2.344210,수육/보쌈
7,25,2.316628,회(사시미)
8,5,2.298451,피자
9,18,2.285019,분식(떡볶이/튀김/순대)
10,35,2.284148,닭갈비


Least Favored 10 Foods
--

In [22]:
df_food_ranking.tail(10)

,foodId,eventStrength,foodName
40,1,2.113050,삼계탕
41,34,2.108990,김밥
42,47,2.099598,베트남 쌀국수
43,9,2.082919,샌드위치
44,14,2.069607,비빔밥
45,20,2.056446,수제비
46,26,2.036338,함박스테이크
47,48,2.032874,월남쌈
48,38,1.968258,컵밥
49,28,1.806685,콩국수


Food Recommender
==

Collaborative Filtering Model
==

This method makes automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on a set of items, A is more likely to have B's opinion for a given item than that of a randomly chosen person.

In [23]:
users_items_pivot_matrix_df = interactions_train_df.pivot(index='userId', 
                                                          columns='foodId', 
                                                          values='eventStrength').fillna(0)

users_items_pivot_matrix_df.head(10)

foodId,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
userId,,,,,,,,,,,,,,,,,,,,,
0,2.321928,0.000000,1.000000,0.000000,2.321928,2.584963,2.321928,2.321928,0.000000,2.584963,...,2.584963,2.000000,1.584963,2.321928,2.584963,2.321928,2.000000,2.000000,1.584963,1.584963
1,0.000000,0.000000,1.584963,1.584963,1.584963,1.584963,2.584963,0.000000,0.000000,0.000000,...,1.584963,1.000000,2.584963,2.584963,1.584963,0.000000,1.584963,0.000000,1.000000,1.584963
2,2.000000,2.321928,1.584963,2.321928,1.584963,1.584963,2.321928,2.000000,0.000000,2.000000,...,0.000000,2.000000,2.000000,1.584963,2.321928,0.000000,2.000000,2.584963,1.584963,2.584963
3,2.584963,0.000000,2.584963,2.000000,2.321928,2.321928,2.584963,2.321928,2.000000,2.321928,...,2.584963,0.000000,2.321928,2.000000,0.000000,2.000000,0.000000,2.321928,0.000000,2.000000
4,2.000000,2.000000,2.000000,2.584963,2.321928,2.584963,0.000000,2.584963,2.321928,2.584963,...,0.000000,0.000000,2.321928,0.000000,2.584963,2.584963,2.321928,2.584963,2.584963,2.584963
5,2.584963,1.584963,1.584963,2.584963,2.321928,2.584963,2.584963,2.584963,2.321928,2.000000,...,0.000000,2.321928,2.321928,2.321928,0.000000,0.000000,1.000000,2.584963,1.584963,1.000000
6,2.584963,2.000000,2.584963,0.000000,2.321928,0.000000,2.321928,2.000000,2.000000,1.584963,...,2.000000,2.000000,2.584963,1.584963,2.321928,0.000000,2.000000,2.321928,2.321928,2.000000
7,0.000000,2.321928,2.321928,2.584963,2.321928,2.321928,0.000000,0.000000,2.321928,2.000000,...,2.584963,2.321928,2.321928,2.321928,2.584963,0.000000,2.321928,2.584963,2.584963,2.321928
8,1.584963,2.000000,2.321928,2.000000,2.321928,2.000000,2.000000,2.000000,2.321928,2.321928,...,2.000000,2.584963,2.321928,0.000000,0.000000,1.584963,2.321928,0.000000,2.321928,2.000000


In [24]:
#The number of factors to factor the user-item matrix.
users_items_pivot_matrix = users_items_pivot_matrix_df.values
NUMBER_OF_FACTORS_MF = 20
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

print('Users items pivot matrix shape : ', users_items_pivot_matrix.shape, 
      '\n\n---------------------------------------------------\n',
      '\nU shape : ', U.shape,
      '\nVt shape : ', Vt.shape,
      '\nSigma : ', sigma.shape,
      '\n\n====================================================',
      '\n\nU*sigma*Vt ='
     '\n\nAll user predicted ratings : ', all_user_predicted_ratings.shape)

Users items pivot matrix shape :  (159, 49) 

---------------------------------------------------
 
U shape :  (159, 20) 
Vt shape :  (20, 49) 
Sigma :  (20, 20) 


U*sigma*Vt =

All user predicted ratings :  (159, 49)


In [30]:
item_ids = food_df['foodId'].tolist()

In [73]:
def interactions_train_test_split(interactions_full_df, test_size=0.20):
    #randomise
    interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['userId'], 
                                   test_size=test_size)
    print('# interactions on Train set: %d' % len(interactions_train_df))
    print('# interactions on Test set: %d' % len(interactions_test_df))
    
    return interactions_train_df, interactions_test_df

interactions_train_df, interactions_test_df = interactions_train_test_split(interactions_full_df, test_size=0.30)

def indexed_df(interactions_full_df,interactions_train_df,interactions_test_df):
    #Indexing by userId to speed up the searches during evaluation
    interactions_full_indexed_df = interactions_full_df.set_index('userId')
    interactions_train_indexed_df = interactions_train_df.set_index('userId')
    interactions_test_indexed_df = interactions_test_df.set_index('userId')
    
    return interactions_full_indexed_df, interactions_train_indexed_df, interactions_test_indexed_df

interactions_full_indexed_df, interactions_train_indexed_df, interactions_test_indexed_df = indexed_df(interactions_full_df,interactions_train_df,interactions_test_df)

def users_items_svd(interactions_df, nfactors =15):
    #Creating a sparse pivot table with users in rows and items in columns
    users_items_pivot_matrix_df = interactions_df.pivot(index='userId', 
                                                          columns='foodId', 
                                                          values='eventStrength').fillna(0)
    
    users_items_pivot_matrix = users_items_pivot_matrix_df.values
    users_ids = list(users_items_pivot_matrix_df.index)
    
    #The number of factors to factor the user-item matrix.
    NUMBER_OF_FACTORS_MF = 15
    
    #Performs matrix factorization of the original user item matrix
    
    U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
    
    sigma = np.diag(sigma)
    
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
    
    #Converting the reconstructed matrix back to a Pandas dataframe
    cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
    
    return cf_preds_df

cf_preds_df = users_items_svd(interactions_train_df, nfactors =15)

# interactions on Train set: 5385
# interactions on Test set: 2308


In [32]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['foodId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'foodId', 
                                                          right_on = 'foodId')[['recStrength', 'foodId', 'foodName']]


        return recommendations_df
    

cf_recommender_model = CFRecommender(cf_preds_df, food_df)


In [36]:
personNum = int(input('userId를 입력해주세요 : '))

cf_model = cf_recommender_model.recommend_items(personNum,verbose=True)

for food in range(len(food_df.foodId)):
    cf_model.loc[cf_model.foodId == food_df.foodId[food],'foodName'] = food_df.foodName[food]

cf_model.index = range(1,len(cf_model)+1)

cf_model

userId를 입력해주세요 :  100


,recStrength,foodId,foodName
1,2.928590,36,불고기
2,2.888081,27,찜 (갈비찜/찜닭 등)
3,2.784266,10,햄버거
4,2.689141,19,된장찌개
5,2.596147,7,"해장국(뼈해장국, 순대국 등)"
6,2.556401,40,칼국수
7,2.517135,9,샌드위치
8,2.502674,39,스테이크
9,2.501507,1,삼계탕
10,2.491436,4,족발


In [37]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
model_evaluator.set_random_sample_non_interacted(0)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
158 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 1.0, 'recall@10': 1.0}


,_person_id,hits@10_count,hits@5_count,interacted_count,recall@10,recall@5
0,43,10,10,10,1.0,1.0
109,86,10,10,10,1.0,1.0
107,48,10,10,10,1.0,1.0
106,19,10,10,10,1.0,1.0
105,64,10,10,10,1.0,1.0
104,82,10,10,10,1.0,1.0
103,42,10,10,10,1.0,1.0
102,49,10,10,10,1.0,1.0
101,99,10,10,10,1.0,1.0
100,13,10,10,10,1.0,1.0


Content-based Filtering
==

This method uses only information about the description and attributes of the items users has previously consumed to model user's preferences. In other words, these algorithms try to recommend items that are similar to those that a user liked in the past (or is examining in the present). In particular, various candidate items are compared with items previously rated by the user and the best-matching items are recommended.



**Model-based approach**

This approach, models are developed using different machine learning algorithms to recommend items to users. There are many model-based CF algorithms, like neural networks, bayesian networks, clustering models, and latent factor models such as Singular Value Decomposition (SVD) and, probabilistic latent semantic analysis.

In [38]:
with open('data\\stopwords-ko.txt', 'r',encoding='utf-8') as f:
    lines = f.readlines()
    stopwords_ko = list(map(lambda x: x.rstrip('\n'),lines))

In [77]:
def tfidf_vectorizer(food_df,stopwords_list ):    
    stopwords_list = stopwords_list
    vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)
    item_ids = food_df['foodId'].tolist()
    
    tfidf_matrix = vectorizer.fit_transform(food_df['foodName'])
    #foodName, foodDescribtion!!!
    tfidf_feature_names = vectorizer.get_feature_names()
    
    return tfidf_matrix, tfidf_feature_names
    

In [78]:
tfidf_matrix, tfidf_feature_names = tfidf_vectorizer(food_df,stopwords_ko)

In [40]:
pd.DataFrame(tfidf_matrix.todense(),columns = tfidf_feature_names).head()

,갈비찜,갈비찜 찜닭,갈비탕,갈비탕 설렁탕,게장,곱창,곱창 막창,구이,김밥,김치찌개,...,콩국수,타코,튀김,튀김 순대,파스타,피자,함박스테이크,해장국,해장국 뼈해장국,햄버거
0,0.0,0.0,0.0,0.0,0.0,0.57735,0.57735,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['foodId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_full_df[interactions_full_df['foodId'] \
                                                   .isin(food_df['foodId'])].set_index('userId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [42]:
user_profiles = build_users_profiles()
len(user_profiles)

159

In [43]:
myuserid = int(input('perosnId를 입력해주세요 : '))
myprofile = user_profiles[myuserid]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        myprofile.flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

perosnId를 입력해주세요 :  10


(1, 81)


,token,relevance
0,게장,0.161114
1,사시미,0.161114
2,초밥,0.161114
3,치킨,0.161114
4,피자,0.161114
5,김밥,0.144720
6,김치찌개,0.144720
7,냉모밀,0.144720
8,닭갈비,0.144720
9,돈까스,0.144720


In [44]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['foodId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'foodId', 
                                                          right_on = 'foodId')[['recStrength', 'foodId', 'foodName']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(food_df)

In [37]:
personNum = int(input('userId를 입력해주세요 : '))

cb_model = content_based_recommender_model.recommend_items(personNum,verbose=True)

for food in range(len(food_df.foodId)):
    cb_model.loc[cb_model.foodId == food_df.foodId[food],'foodName'] = food_df.foodName[food]

cb_model.index = range(1,len(cb_model)+1)

cb_model

userId를 입력해주세요 :  90


,recStrength,foodId,foodName
1,0.16115,27,볶음밥
2,0.16115,5,돈까스
3,0.16115,36,회(사시미)
4,0.16115,35,샌드위치
5,0.16115,34,닭갈비
6,0.16115,33,제육볶음
7,0.16115,16,피자
8,0.16115,39,스테이크
9,0.16115,3,브리또&타코
10,0.16115,14,곱창/막창


In [45]:
model_evaluator.set_random_sample_non_interacted(0)
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
158 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 1.0, 'recall@10': 1.0}


,_person_id,hits@10_count,hits@5_count,interacted_count,recall@10,recall@5
0,43,10,10,10,1.0,1.0
109,86,10,10,10,1.0,1.0
107,48,10,10,10,1.0,1.0
106,19,10,10,10,1.0,1.0
105,64,10,10,10,1.0,1.0
104,82,10,10,10,1.0,1.0
103,42,10,10,10,1.0,1.0
102,49,10,10,10,1.0,1.0
101,99,10,10,10,1.0,1.0
100,13,10,10,10,1.0,1.0


여러명이서 같이 먹을 떄
--

In [46]:
def joint_recommender_model(recommender_model, *args):
    
    model = recommender_model
    
    model_list=[]
    userId_list=[]
    
   #find the first common food shared by people searching from top 1 to the last.
    
    top=1
    common_content = []
    while common_content == []:
        
        userId_list = []
        model_list = []
        user1_content=[]
        common_content = []
        
        
        for arg in args :
            
            userId_list.append(arg)
            model_list.append(model.recommend_items(arg,verbose=True,topn=top))
            
        user1_content = list(model_list[0].foodId)
        common_content = user1_content
        
        for user in model_list:
            user_content = list(user.foodId)
            common_content = list(set(common_content).intersection(user_content))
        
        
        if common_content == []:
            top=top+1
        else: 
            break
            
    
    
    joint_content_df = food_df[food_df.foodId == common_content]
    
    print(len(userId_list),'명이 함께 먹을 추천 음식은 [',list(joint_content_df.foodName)[0],']입니다!!!\n')
    
    person = 0
    
    for user in model_list:
        rank = user[user.foodId == common_content].index[0] + 1
        
        print('UserId ', userId_list[person],'번은 이 음식을', rank, '번 째로 좋아합니다!')
        
        person = person + 1 

In [49]:
#함꼐 먹을 때 음식을 추천해주는 알고리즘

rec_user_list = []
get_user = '1'

while True:
    while get_user in (list(map(lambda x: str(x), list(df_user_info.userId)))+['END']):
        get_user = input('\nuserId를 입력해주세요(종료는 END) : ')
        
        if get_user in list(map(lambda x: str(x), rec_user_list)):
            while get_user in list(map(lambda x : str(x), rec_user_list)):
                print('\n\n[[[중복된 userId를 입력하였습니다!!!]]]\n')
                get_user = input('제대로 된 userId를 입력해주세요(종료는 END) : ')
        
        if get_user not in (list(map(lambda x: str(x), list(df_user_info.userId)))+['END']):
            while get_user not in (list(map(lambda x: str(x), list(df_user_info.userId)))+['END']):
                print('\n\n[[[잘못된 UserId를 입력하였습니다!!!]]]\n')
                get_user = input('제대로 된 userId를 입력해주세요(종료는 END) : ')
                
                if get_user in list(map(lambda x: str(x), rec_user_list)):
                    while get_user in list(map(lambda x : str(x), rec_user_list)):
                        print('\n\n[[[중복된 userId를 입력하였습니다!!!]]]\n')
                        get_user = input('제대로 된 userId를 입력해주세요(종료는 END) : ')
        
        
        if get_user in list(map(lambda x: str(x), list(df_user_info.userId))):
            rec_user_list.append(int(get_user))
        
        if get_user == 'END':
            break
    break
        
        
            
    rec_user_list.append(get_user)
    
print('\n\n---------------------------------------------------------------------------\n\n')

cf_recommender_model_random = CFRecommenderRandom(interactions_full_df, food_df)

joint_recommender_model(cf_recommender_model_random ,*rec_user_list)


userId를 입력해주세요(종료는 END) :  10

userId를 입력해주세요(종료는 END) :  2

userId를 입력해주세요(종료는 END) :  END




---------------------------------------------------------------------------


2 명이 함께 먹을 추천 음식은 [ 샤브샤브 ]입니다!!!

UserId  10 번은 이 음식을 5 번 째로 좋아합니다!
UserId  2 번은 이 음식을 2 번 째로 좋아합니다!


In [50]:

#Add new interaction
def new_interaction(old_interaction_df, userId, foodId, eventStrength):
    new_interaction_df = pd.DataFrame([[userId,foodId,eventStrength]],columns = ['userId','foodId','eventStrength'])
    old_interaction_df = old_interaction_df.append(new_interaction_df,ignore_index=True)
    return old_interaction_df

#Add new food
def new_food(old_food_df,foodId,foodName):
    new_food_df = pd.DataFrame([[foodId,foodName]], columns = ['foodId','foodName'])
    old_food_df = old_food_df.append(new_food_df,ignore_index=True)
    return old_food_df


#Add new users
def new_user_info(old_user_info_df, userId,timestamp,userName,sex,age,aloneHow,eatAlone,eatDate,eatTogether):
    new_user_info_df = pd.DataFrame([[userId,timestamp,userName,sex,age,aloneHow,eatAlone,eatDate,eatTogether]], 
                                    columns = ['userId','timestamp','userName','sex','age','aloneHow','eatAlone','eatDate','eatTogether'])
    old_user_info_df = old_user_info_df.append(new_user_info_df,ignore_index=True)

In [55]:
def cold_start_question(userId,number):
    foodIdlist = food_df.foodId.tolist()
    selected_content = np.random.choice(foodIdlist,number,replace=False )
    
    question = []
    
    for foodId in selected_content:
        food = food_df.foodName[food_df.foodId == foodId].to_string(header=False,index=False)
        eventStrength = ''
        
        while eventStrength not in ['0','1','2','3','4','5']:
            eventStrength = input('\n당신은 ' + food+'을(를) 얼마나 좋아하십니까?\n\n0 - 먹어본 적 없음\n1 - 별로 좋아하지 않음\n5 - 매우 좋아함\n\n')
            
            if eventStrength not in ['0','1','2','3','4','5']:
                print('\n\n0-5 사이의 숫자릅 입력하여 주세요!')
            
        eventStrength = int(eventStrength)
            
        question.append([userId,foodId,eventStrength])
        
    starting_question_df = pd.DataFrame(question,columns = ['userId','foodId','eventStrength'])
    
    return starting_question_df


In [56]:
cold_start_question(1000,5)


당신은 돈까스을(를) 얼마나 좋아하십니까?

0 - 먹어본 적 없음
1 - 별로 좋아하지 않음
5 - 매우 좋아함

 0

당신은 치킨을(를) 얼마나 좋아하십니까?

0 - 먹어본 적 없음
1 - 별로 좋아하지 않음
5 - 매우 좋아함

 5

당신은 월남쌈을(를) 얼마나 좋아하십니까?

0 - 먹어본 적 없음
1 - 별로 좋아하지 않음
5 - 매우 좋아함

 4

당신은 삼계탕을(를) 얼마나 좋아하십니까?

0 - 먹어본 적 없음
1 - 별로 좋아하지 않음
5 - 매우 좋아함

 2

당신은 리조또을(를) 얼마나 좋아하십니까?

0 - 먹어본 적 없음
1 - 별로 좋아하지 않음
5 - 매우 좋아함

 1


,userId,foodId,eventStrength
0,1000,33,0
1,1000,24,5
2,1000,48,4
3,1000,1,2
4,1000,17,1
